<center>
    <img width='80%' src='rapids_workflow.png'></center>    


<h1>RAPIDS Demo - End-To-End ML Workflow - Dask Early Draft </h1>

In [ ]:
import pandas as pd, numpy as np, sklearn
from sklearn import datasets
import cudf, numba, scipy
import time

In [ ]:
import dask
import dask.array, dask.dataframe
from dask import delayed
import dask_cudf
from dask.diagnostics import ResourceProfiler #Profiler, CacheProfiler

In [ ]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client

In [ ]:
cluster = LocalCUDACluster()
client = Client(cluster)

In [ ]:
client

-----
# 2. Generate Dataset [ X: features, y: labels ]

Set the size of the generated dataset -- the number of total samples is determined by this value

In [ ]:
nTotalSamples = 100000000

Next we'll use <a href='https://scikit-learn.org/stable/datasets/index.html#generated-datasets'>sklearn.datasets</a> to build synthetic sub-datasets of the size we specified above. We'll build three sub-datasets and combine them together and then use a trained model to see if we can determine which of sub-dataset a sample belongs to. The three sub-datasets are built using the moons, blobs, and swiss-roll generators. These sub-datasets were selected for their distinct visual features.

# Dask [ sharded data gen & cudf conversion ]

In [ ]:
nShards = 4

nDatasets = 3

nSamplesPerShardSubset = (nTotalSamples//nShards)//nDatasets
nSamplesPerShard = nSamplesPerShardSubset*nDatasets
nDatasetCols = 4

In [ ]:
nSamplesPerShard, nSamplesPerShardSubset

In [ ]:
def generate_data_shard ( nSamplesPerShardSubset ):
    
    # generate features
    swissRollDataset = datasets.make_swiss_roll( n_samples = nSamplesPerShardSubset, noise = .005)[0]

    moonsDataset = datasets.make_moons(n_samples = nSamplesPerShardSubset, noise = 0)[0]
    moonsDataset = np.hstack( [moonsDataset, np.zeros( (moonsDataset.shape[0], 1) )] )*5

    blobsDataset = datasets.make_blobs( n_samples = nSamplesPerShardSubset, centers = 5,  n_features = 3, 
                                        cluster_std = 0.25,  random_state = 0)[0] + [0, 1.5, 0]
    
    # generate labels for classification 
    blobsLabels = np.zeros(blobsDataset.shape[0])
    moonsLabels = 1 * np.ones(moonsDataset.shape[0])
    sRollLabels = 2 * np.ones(swissRollDataset.shape[0])

    features = np.vstack([blobsDataset, moonsDataset, swissRollDataset])
    labels = np.hstack( [blobsLabels, moonsLabels, sRollLabels] )
    
    return np.asfortranarray(np.hstack( [ features, np.expand_dims(labels,axis=1) ] ))

# Define Dask Compute Graph

In [ ]:
delayedDataShards = [ delayed(generate_data_shard)(nSamplesPerShardSubset) for iShard in range(nShards) ]
delayedDataFrames = [ delayed(pd.DataFrame)(iShard) for iShard in delayedDataShards ]
delayedGPUDataFrames = [ delayed(cudf.from_pandas)(iShard) for iShard in delayedDataFrames]
mergedGPUDataFame = delayed(cudf.concat)(delayedGPUDataFrames)

In [ ]:
mergedGPUDataFame.visualize()

In [ ]:
tic = time.time()
with ResourceProfiler(dt=0.25) as rprof:
    out = mergedGPUDataFame.compute()
print( time.time() - tic )

In [ ]:
rprof.results

# Non-Dask Dataset Generation

In [ ]:
%%time
nSamplesPerSubDataset = nTotalSamples//3

swissRollDataset = datasets.make_swiss_roll( n_samples = nSamplesPerSubDataset, noise = .005)[0]

moonsDataset = datasets.make_moons(n_samples = nSamplesPerSubDataset, noise = 0)[0]
moonsDataset = np.hstack( [moonsDataset, np.zeros( (moonsDataset.shape[0], 1) )] )*5

blobsDataset = datasets.make_blobs( n_samples = nSamplesPerSubDataset, centers = 5,  n_features = 3, 
                                    cluster_std = 0.25,  random_state = 0)[0] + [0, 1.5, 0]

X = np.vstack([blobsDataset, moonsDataset, swissRollDataset])

# generate labels for classification 
blobsLabels = np.zeros(blobsDataset.shape[0])
moonsLabels = 1 * np.ones(moonsDataset.shape[0])
sRollLabels = 2 * np.ones(swissRollDataset.shape[0])

y = np.hstack( [blobsLabels, moonsLabels, sRollLabels] )

In [ ]:
X.shape, y.shape

## 2.1 - Split Train (75%) and Test (25%) Data 
We split our combined dataset into two portions:
* **train-set** - which we'll use to optimize our model's parameters [ train-set = randomly selected 75% of total data]
* **test-set** - which we'll use to evaluate how well our trained model performs on unseen data [ test-set = remaining 25% of data ]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.25, random_state = 0, shuffle=True)

In [ ]:
len(X_train), len(X_test)

## 2.2 - Visualize Data
We define a function for WebGL based 3D plotting using the <a href='https://github.com/maartenbreddels/ipyvolume'>ipyvolume</a> library -- we restrict the maximum points to plot to `maxSamplesToPlot` which has a default setting of `100000`

In [ ]:
def ipv_plot_data( data, colorStack = 'purple', 
                  maxSamplesToPlot = 100000, 
                  holdOnFlag = False, markerSize=.5):
    
    nSamplesToPlot = np.min( ( len(data), maxSamplesToPlot ) )
    if not holdOnFlag: ipv.figure(width=600,height=600)
        
    if isinstance(colorStack, np.ndarray):
        colorStack = colorStack[0:maxSamplesToPlot,:]

    ipv.scatter( data[0:nSamplesToPlot,0], 
                 data[0:nSamplesToPlot,1], 
                 data[0:nSamplesToPlot,2], size = markerSize, 
                 marker = 'sphere', color = colorStack)
    
    if not holdOnFlag: ipv.show()


## Sub-Datasets [ moons, blobs, swiss-roll ] 

In [ ]:
ipv_plot_data( moonsDataset)

In [ ]:
ipv_plot_data( blobsDataset )

In [ ]:
ipv_plot_data( swissRollDataset )

## Combined Dataset Plot
The train-set is shown in purple and the test-set is show in yellow.

In [ ]:
%time
ipv.figure()
ipv_plot_data( X_train, 'purple', 200000, True)
ipv_plot_data( X_test, 'yellow', 1000, True, 1)
ipv.show()

------
# 3. ETL 
First we write the dataset to disk (as a comma separated file - CSV) so that we can subsequently demonstrate data loading.

In [ ]:
%%time
pd.DataFrame(data = X_train).to_csv('X_train.csv.txt', index = False)
pd.DataFrame(data = X_test).to_csv('X_test.csv.txt', index = False)
pd.DataFrame(data = y_train).to_csv('y_train.csv.txt', index = False)
pd.DataFrame(data = y_test).to_csv('y_test.csv.txt', index = False)

In [ ]:
!echo 'no data\n0' > warmup.csv # write a mini csv file used to initialize cuIO kernels

## Check Size of Data on Disk 
using the default value of `nTotalSamples = 5000000` should produce a training set of `~184MBs` in size.

In [ ]:
!du -h *csv.txt

# 3.1 - Load Data

In [ ]:
%%time 
startTime = time.time()

pd_X_train = pd.read_csv('X_train.csv.txt',  delimiter=',')
pd_X_test = pd.read_csv('X_test.csv.txt',  delimiter=',')
pd_y_train = pd.read_csv('y_train.csv.txt',  delimiter=',')
pd_y_test = pd.read_csv('y_test.csv.txt',  delimiter=',')

pandasIngestionTime = time.time() - startTime

In [ ]:
# get column-names
f = open('X_train.csv.txt'); colNames = f.readline().strip().split(','); f.close()
# warmup rapids data ingestion engines [ cuio kernels ]
cudf.read_csv('warmup.csv')

In [ ]:
%%time
startTime = time.time()

cudf_X_train = cudf.read_csv('X_train.csv.txt', delimiter=',', skiprows=1, names=colNames, dtype=['float64', 'float64', 'float64'])
cudf_X_test = cudf.read_csv('X_test.csv.txt', delimiter=',', skiprows=1, names=colNames, dtype=['float64', 'float64', 'float64'])
cudf_y_train = cudf.read_csv('y_train.csv.txt', dtype=['float64'])
cudf_y_test = cudf.read_csv('y_test.csv.txt', dtype=['float64'])

rapidsIngestionTime = time.time() - startTime

## Evaluate Load/Ingestion Speedup

In [ ]:
pandasIngestionTime/rapidsIngestionTime

# 3.2 - Transform Data ( Normalize )
Transforming a dataset is a common requirement prior to training upstream models. For each feature in the dataset we remove the mean and divide by the standard deviation -- this makes each feature behave like a normally distributed variable (e.g. gaussian with 0 mean and unit variance). 

For the data on the CPU we can use the pre-built <a href='https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html'>sklearn.preprocessing.StandardScaler</a> function.
In the case of the GPU, we demonstrate how the same transformation can be built using a custom (user defined) function written as a <a href='http://numba.pydata.org/numba-doc/0.13/CUDAJit.html'>just-in-time numba kernel</a>. 

Note that we compute the mean and standard deviation statistics on the training data, and then apply the transformation to the training and test data (i.e., the test data is never seen when computing the mean & standard deviation).

In [ ]:
%%time 
startTime = time.time()

scaler = sklearn.preprocessing.StandardScaler().fit(pd_X_train) # normalize
pd_X_train = scaler.transform(pd_X_train)
pd_X_test = scaler.transform(pd_X_test)

pandasTransformTime = time.time() - startTime 

-----

In [ ]:
@cuda.jit 
def gpu_scale(outputCol, colGPUArrays, colMeans, colStDevs):
    iRow = cuda.grid(1)
    if iRow < colGPUArrays.size:
        outputCol[iRow] = ( colGPUArrays[iRow] - colMeans ) / ( colStDevs + 1e-10 )

In [ ]:
def standard_scaler_numba( targetDF, trainMeans = None, trainStdevs = None):
    nRows = targetDF.shape[0]
    
    blockSize = 128
    blockCount = nRows // blockSize + 1
    scaledDF = cudf.DataFrame()
    
    if trainMeans is None and trainStdevs is None:
        trainMeans = {}
        trainStdevs = {}
        
    for iColName in targetDF.columns:
        colGPUArray = targetDF[iColName].to_gpu_array()
        outputCol = cuda.device_array ( shape=(nRows), dtype=colGPUArray.dtype.name)       
        if iColName not in trainMeans.keys():
            trainMeans[iColName] = targetDF[iColName].mean()
        if iColName not in trainStdevs.keys():
            trainStdevs[iColName] = targetDF[iColName].std()
        gpu_scale[(blockCount),(blockSize)](outputCol, colGPUArray, trainMeans[iColName], trainStdevs[iColName])
        scaledDF.add_column(name=iColName, data = outputCol)    
        
    return scaledDF, trainMeans, trainStdevs

In [ ]:
_, _, _ = standard_scaler_numba( cudf_X_test.copy().head(2) ) # warmup

In [ ]:
%%time
startTime = time.time()

cudf_X_train, trainMeans, trainStdevs = standard_scaler_numba( cudf_X_train )
cudf_X_test, _, _ = standard_scaler_numba( cudf_X_test, trainMeans, trainStdevs )

rapidsTransformTime = time.time() - startTime

## Evaluate Transform Speedup

In [ ]:
pandasTransformTime/rapidsTransformTime

## Verify [approximate] numerical equivalence between CPU and GPU

In [ ]:
trainMeans, scaler.mean_, trainStdevs, scaler.scale_

In [ ]:
print(pd_X_test[0:2,:])

In [ ]:
print(cudf_X_test.head(2))

-----
# 4. - Model Building with XGBoost
-----
XGBoost is a popular algorithm for classification. It uses a sequence of decision trees built in succession such that each new tree attempts to correct the errors made by its predecessors (analogy to multiple golf swings [ each improving on the past ] to reach a target). For a deeper dive into how XGBoost works check out the following dev blog: <br>
> https://devblogs.nvidia.com/gradient-boosting-decision-trees-xgboost-cuda/

<img src='xgboost.png' width =90%>
<center> img src: https://explained.ai/gradient-boosting/L2-loss.html </center>

## Prepare Data for Training

In [ ]:
X = pd_X_train
y = pd_y_train

In [ ]:
X_gpu = xgboost.DMatrix(pd_X_train, label=np.squeeze(pd_y_train))
y_gpu = xgboost.DMatrix(pd_X_test, label=np.squeeze(pd_y_test))

## Specify Model Parameters

noteable parameters: [ to see all available options execute '?xgboost.XGBClassifier' in a new cell] 

* __max_depth__ : int [ default = 3 ] -- Maximum tree depth for base learners.
* __n_estimators__ : int [ default = 100 ] -- Number of boosted trees to fit.
* __n_jobs__ : int [ default = 1 ] -- Number of parallel threads used to run xgboost.

In [ ]:
cpuMaxDepth = 3 # default
nTrees = 100 # default

gpuMaxDepth = 10

nCores = !nproc --all
nCores = int(nCores[0])

paramsGPU = {
    'max_depth': gpuMaxDepth,
    'n_estimators': nTrees,
    'n_gpus': 1,
    'tree_method': 'gpu_hist',
    'objective': 'gpu:reg:linear', 
    'random_state': 0,
    'verbose_model': True
}
paramsCPU = {
    'max_depth': cpuMaxDepth,
    'n_estimators': nTrees,
    'tree_method': 'hist',
    'objective': 'binary:logistic',
    'n_jobs': nCores
}

-----
# 4.1 - Model Training

## Train on **CPU**

In [ ]:
xgBoostModelCPU = xgboost.XGBClassifier(max_depth = paramsCPU['max_depth'], 
                                        n_estimators = paramsCPU['n_estimators'],
                                        tree_method = paramsCPU['tree_method'],
                                        objective = paramsCPU['objective'],                                        
                                        n_jobs = paramsCPU['n_jobs'])

In [ ]:
xgBoostModelCPU, print("using {} CPU cores for parallel xgboost training".format(nCores))

In [ ]:
%%time
startTime = time.time()

xgBoostModelCPU.fit( X, np.squeeze(y) );

cpuXGBoostTime = time.time() - startTime

## Train on **GPU**

In [ ]:
%%time
startTime = time.time()

xgBoostModelGPU = xgboost.train( dtrain = X_gpu, params = paramsGPU)

gpuXGBoostTime = time.time() - startTime

## Evaluate Training Speedup

In [ ]:
cpuXGBoostTime/gpuXGBoostTime

-----
# 4.2 - Model Inference

## Infer/predict using Trained **CPU** Model

In [ ]:
%%time
startTime = time.time()

yPredTrain = xgBoostModelCPU.predict(pd_X_train)
yPredTest = xgBoostModelCPU.predict(pd_X_test)

cpuXGBoostInferenceTime = time.time() - startTime

## Infer/predict using Trained **GPU** Model
> note that our objective was changed to a regression [ gpu accelerated ] so we must take care to convert each of our predictions from a continuous value to a discrete class (essentially by rounding).

In [ ]:
# convert continuous prediction to a multi-class option
def continuous_to_discrete( data, nClasses = 3):
    data[data>nClasses-1] = nClasses-1 # filter values beyond the possible classes
    return np.abs(np.round(data))

In [ ]:
%%time
startTime = time.time()

yPredTrain_GPU = continuous_to_discrete ( xgBoostModelGPU.predict(X_gpu) )
yPredTest_GPU = continuous_to_discrete ( xgBoostModelGPU.predict(y_gpu) )

gpuXGBoostInferenceTime = time.time() - startTime

## Evaluate Inference Speedup

In [ ]:
cpuXGBoostInferenceTime/gpuXGBoostInferenceTime

-----
# 4.3 - Evaluate Accuracy

In [ ]:
print( 'CPU test accuracy: {0:.6f} '.format( accuracy_score(pd_y_test, yPredTest) ))
print( 'GPU test accuracy: {0:.6f} '.format( accuracy_score(pd_y_test, yPredTest_GPU) ))

TODO: to increase model accuracy, increase complexity, number of trees, max_depth


In [ ]:
print('\n confusion matrix on TRAIN data -- ')
print(confusion_matrix(pd_y_train, yPredTrain))
print('\n confusion matrix on TEST data -- ')
print( confusion_matrix(pd_y_test, yPredTest))

In [ ]:
print('\n confusion matrix on TRAIN data -- ')
print(confusion_matrix(pd_y_train, yPredTrain_GPU))
print('\n confusion matrix on TEST data -- ')
print( confusion_matrix(pd_y_test, yPredTest_GPU))

# 4.4 - Visualize Model Outputs

## Visualizing a CPU boosted tree

In [ ]:
fig = plt.figure(figsize=(50,50))
plot_tree(xgBoostModelCPU, num_trees=0, ax=plt.subplot(1,1,1))

## Visualizing a GPU Boosted Tree

In [ ]:
fig = plt.figure(figsize=(50,50))
plot_tree(xgBoostModelGPU, num_trees=0, ax=plt.subplot(1,1,1))

## Visualize Class Predictions

In [ ]:
def map_colors_to_clusters_topK ( dataset, labels, topK=None, cmapName = 'tab10'):
    if topK == None:
        topK = dataset.shape[0]
    
    colorStack = np.zeros((topK, 3), dtype=np.float32)
    
    cMap = plt.get_cmap(cmapName)
    for iColor in range ( topK ):
        colorStack[iColor] = cMap.colors[ labels[iColor] ]
        
    return colorStack    

In [ ]:
colorStackClassifier = map_colors_to_clusters_topK ( pd_X_test, yPredTest_GPU.astype(np.int), topK=None )

In [ ]:
ipv_plot_data( pd_X_test, colorStack= colorStackClassifier)

-------
# Extensions
-------

# Ext.1 : Growing the Model Ensemble ( DBScan Clustering )

<img width='95%' src='https://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_0011.png'>

## DBScan via sklearn [ CPU ]

In [ ]:
# sub-sample
nSamplesToCluster = 30000
pd_X_test_sampled = pd_X_test[:nSamplesToCluster, :]  
cudf_X_test_sampled = cudf_X_test.loc[0:nSamplesToCluster,list(cudf_X_test.columns)]

In [ ]:
%%time
startTime = time.time()

dbScanModel = DBSCAN( min_samples = 10, n_jobs = nCores ).fit(pd_X_test_sampled)
labels = dbScanModel.labels_

sklearnDBScanTime = time.time() - startTime

## DBScan via cuml [ GPU ]

In [ ]:
%%time
startTime = time.time()

clustering_cuml = cuml.DBSCAN( eps = .15 , min_samples = 200 )
clustering_cuml.fit( cudf_X_test_sampled )

rapidsDBScanTime = time.time() - startTime

In [ ]:
%%time
print( clustering_cuml.labels_.value_counts() )

## Evaluate Clustering Speedup

In [ ]:
sklearnDBScanTime/rapidsDBScanTime

## Map Clusters to Colors and Visualize

In [ ]:
%%time
colorStack = map_colors_to_clusters_topK( pd_X_test, clustering_cuml.labels_, nSamplesToCluster  )

In [ ]:
def colored_topK ( dataset, topK = None):    
    if topK == None:
        topK == dataset.shape[0]
        
    return lambda dataset, colorStack, topK : ipv.quickscatter( dataset[0:topK, 0], 
                                                                dataset[0:topK, 1], 
                                                                dataset[0:topK, 2], 
                                                                size = .5, marker = 'sphere', 
                                                                color = colorStack)  

In [ ]:
colored_topK( pd_X_test_sampled, nSamplesToCluster )(pd_X_test_sampled, colorStack, nSamplesToCluster)

### [ TODO ] Ext.2 : cross-validation
### [ TODO ] Ext.3 : hyper-parameter search

-----
# End [ thanks! ]

please provide feedback/suggestions and errata @ https://github.com/miroenev/rapids